# 8 Queens

### Introduction

<figure>
<img src="resources/eight_queens_moves.png", width=300 align="right">
    <figcaption></figcaption>
</figure>

The [Eight Queens puzzle](https://en.wikipedia.org/wiki/Eight_queens_puzzle) is a famous puzzle that has been studied extensively in- and outside of computer science. It was first published in the chess magazine _Schach_ in 1848. 

The problem can be formulated as follows: 

_"Place 8 queens on a regular (8x8) chess board such that no queen attacks any other queen."_

A queen in the game of chess can move horizontally, vertically, and diagonally. The puzzle can be solved by hand (and even [Carl Friedrich Gauss](https://en.wikipedia.org/wiki/Carl_Friedrich_Gauss) studied it back in 1850).

The EightQueensState class below, as well as the methods defined, should prove a helpful start for a Genetic Algorithms approach. However, you are welcome to change as little or as much of the code as is useful.

In [1]:
import numpy as np

class EightQueensState:
    """This class represents a board in the eight queens puzzle"""
    def __init__(self, state=None, n=8, seed = None):
        """
        :param state: pass in a numpy array of integers to set the state, otherwise will be generated randomly
        :param n: only used if state is not provided, determines size of board (default: 8)
        """
        
        if state is None:
            self.seed = seed
            np.random.seed(seed)
            self.n = n
            state = np.random.randint(0, n, n)
            
        else:
            self.n = len(state)
        self.state = state

    @staticmethod
    def copy_replace(state, i, x):
        """This creates a copy of the state (important as numpy arrays are mutable) with column i set to x"""
        new_state = state.copy()
        new_state[i] = x
        return new_state

    @staticmethod
    def range_missing(start, stop, missing):
        """
        This creates a list of numbers with a single value missing
        e.g. range_missing(0, 8, 2) -> [0, 1, 3, 4, 5, 6, 7]
        """
        return list(range(start, missing)) + list(range(missing + 1, stop))
    
    
    def cost(self):
        """Calculates the number of pairs attacking"""
        count = 0
        for i in range(len(self.state) - 1):
            count += (self.state[i] == np.array(self.state[i + 1:])).sum()

            upper_diagonal = self.state[i] + np.arange(1, self.n - i)
            lower_diagonal = self.state[i] - np.arange(1, self.n - i)
            count += (np.array(self.state[i + 1:]) == upper_diagonal).sum()
            count += (np.array(self.state[i + 1:]) == lower_diagonal).sum()
        return count
    
    def neighbourhood(self):
        """This generates every state possible by changing a single queen position"""
        neighbourhood = []
        for column in range(self.n):
            for new_position in self.range_missing(0, self.n, self.state[column]):
                new_state = self.copy_replace(self.state, column, new_position)
                neighbourhood.append(EightQueensState(new_state))

        return neighbourhood

    def random_neighbour(self):
        """Generates a single random neighbour state, useful for some algorithms"""
        column = np.random.choice(range(self.n))
        new_position = np.random.choice(self.range_missing(0, self.n, self.state[column]))
        new_state = self.copy_replace(self.state, column, new_position)
        return EightQueensState(new_state)
 
    def is_goal(self):
        return self.cost() == 0
    
    # Numpy output to pass it inside ga. 
    def call_array(self):
        return np.array(self.state)
    
    def __len__(self):
        return len(self.state)
    
    def __str__(self):
        if self.is_goal():
            return f"Goal state! {self.state}"
        else:
            return f"{self.state}"

In [2]:
game = EightQueensState()
print(game)
print(game.is_goal())

[1 3 0 1 0 0 7 2]
False


### TASK 1: Solve 8 Queens problem with Genetic Algorithm

More details below in code comments
Print statements are saved also as comments in case more clarification is needed.

In [3]:
import random
import timeit
import warnings

warnings.filterwarnings("ignore", category=DeprecationWarning) 

def genetic_algorithm(n):
    # n is the population size we would like to start with 
    """
    Genetic algorithm for the 8 Queens problem
     Input: n (Population size), must be even
     Returns either a goal state or a local maximum state based on time restrictions 
    """ 
    ga_start = timeit.default_timer()
    
    population = []
    
    def fitness(indiv):
        """Calculates the fitness function for the genetic algorithm""" 
        count = 0
        # For each queen, evaluates the position of the queens on the right hand side, +1 for each conflicting queen
        
        for i in range(len(indiv) - 1):
            # Row evaluation
            count += (indiv[i] == np.array(indiv[i + 1:])).sum()

            # Upper and lower diagonal evaluation
            upper_diagonal = indiv[i] + np.arange(1, 8 - i)
            lower_diagonal = indiv[i] - np.arange(1, 8 - i)
            count += (np.array(indiv[i + 1:]) == upper_diagonal).sum()
            count += (np.array(indiv[i + 1:]) == lower_diagonal).sum()

        # Count only calculates the queens but calculation is based on attacks
        # Each pair of conflicting queens mean 2 attacks, therefore count is multiplied by 2
        # Max attacks per game is 7 and for each queen (8 queens * 7 attacks)
        fitness_score = (56-(count*2))/2
            
        return fitness_score
        
    def is_goal(indiv):
        
        fitness_score = fitness(indiv)
        
        if fitness_score == 28:
            return True

        else:
            return False
    
    # Fixing the seed as stated in the forum to evaluate the algorithm
    seed = 71 #460,530,71
    for i in range(n):
        game = EightQueensState(seed = seed)
        population.append(game.call_array())
        #print(f" Parent {i} = {population[i]} with fitness {fitness(population[i])}.")
        seed +=1
    #print() 
    
    # Randomly tupling the population list values for reproduction
    def reproduce(population):
        """
        Reproduction of the population:
            Tuple parents in pairs
            Crossover based on longest non-conflicting sequences
            Mutate based on a random probability
        Return new_population 
        """
        
        def non_conflict(parent):   
            """
            Finds the lenght of the non-conflicting sequence within a parent. 
            """       
            index_row = 0
            index_diag = 0
            count=0
            
            # For each parent, checks the left hand side to return non-conflicting index 
            
            for i in range(parent.size):
                
            # Row evaluation
                if parent[i] in np.array(parent[:i]):
                    break
                else:
                    index_row = i

            for i in range(parent.size):

            # Lower and upper diagonal evaluation
                upper_diagonal = (parent[i] + np.arange(1, (len(parent) - i)))[:i]
                lower_diagonal = (parent[i] - np.arange(1, len(parent) - i))[:i]
                
                count += np.count_nonzero(np.array(parent[:i]) == upper_diagonal)
                count += np.count_nonzero(np.array(parent[:i]) == lower_diagonal)
        
                if count == 1:
                    index_diag = i
                
                if bool(set(np.array(parent[:i])).intersection(upper_diagonal))== False and bool(set(np.array(parent[:i])).intersection(lower_diagonal))== False:
                    index_diag = i
                else:
                    break
                    
            return min(index_row,index_diag)
        
        
        parents = []
        new_population = []
        
        #print('Randomly matched parents for cross-over:')
        
        # Randomly matches the parents to start re-production
        while len(population)>0:
            x = population.pop(random.randrange(len(population)))
            y = population.pop(random.randrange(len(population)))
            parents.append((x,y)) 
           #print(x,y)
       #print()
        
        for i in range(len(parents)):
            
            # To find optimal crossover lenght to have more efficient crossovers
            # Takes the max since this keeps bigger non-conflicting chunks for at least one parent
            # If swap index is smaller than 1, a random number is selected to keep bigger crossover chunks
            
            index_par1 = non_conflict(parents[i][0])
            index_par2 = non_conflict(parents[i][1])
                
            swap_leng = max(index_par1, index_par2)
            if swap_leng <1:
                swap_leng = random. randint(1,6)
            
             # Crossover
            child1 = np.append(parents[i][0][:swap_leng+1],parents[i][1][swap_leng+1:])
            new_population.append(child1)
            child2 = np.append(parents[i][1][:swap_leng+1],parents[i][0][swap_leng+1:])
            new_population.append(child2)
            
            #print(f"Offspring 1 from pair {i} = {child1}, crossover lenght is {swap_leng+1}.")
            #print(f"Offspring 2 from pair {i} = {child2}, crossover lenght is {swap_leng+1}.")
            #print()
            
        # Hassanat, A., Almohammadi, K., Alkafaween, E., Abunawas, E., Hammouri, A. and Prasath, V., 2019. 
        # Choosing Mutation and Crossover Ratios for Genetic Algorithms—A Review with a New Dynamic Approach. 
        # Information, 10(12), p.390.
        
        # Based on the research above, 0.001 is an optimal mutation rate for populations in range of [50–100]
        # and lower for smaller populations. 
        # Since the algorithm will be evaluated based on the solution, a very low probability is choosen
        # to reduce randomness factor.
    
            mutation_rate = 0.00001
            for child in new_population:
                if random.random() < mutation_rate:
                    mutation_index = random.randint(0,7)
                    child[mutation_index] = random.randint(swap_leng,7)
                    #print(f"Mutation in children {child}.")
                    #print()
                         
        return new_population
    
    
    
    
    goal_state = [item for item in population if is_goal(item)== True]
    
    fitness_list = [(item, fitness(item)) for item in population]
    local_max = max(fitness_list , key = lambda i: i[1])[0]
    
    
    if len(goal_state) != 0:
        return goal_state[0]
    
    while len(goal_state)==0:
        
        ga_current = timeit.default_timer()
        time_total = ga_current-ga_start
        
        if time_total > 35:
            return f"Local maximum is found:{local_max} with fitness score {fitness(local_max)}."
        
        new_population = reproduce(population)
        population = new_population
        
        goal_state = [item for item in population if is_goal(item)== True]
        
        if len(goal_state) != 0:
            return goal_state[0].tolist()
        
        for i in range(len(population)):
            if fitness(population[i])> fitness(local_max):
                local_max = population[i]
            

In [4]:
genetic_algorithm(10)

[3, 5, 7, 1, 6, 0, 2, 4]

## TASK 2: n-Queens Problem

Code is adapted for n-Queens problem

In [5]:
import random
import timeit
import warnings

warnings.filterwarnings("ignore", category=DeprecationWarning) 

def genetic_algorithm_nqueens(n, queens): 
    """
    Genetic algorithm for the n-Queens problem
    Input: n (Population size, must be even), number of queens, number of queens for n-queens
    Returns either a goal state or a local maximum state based on time restrictions 
    """ 
    
    ga_start = timeit.default_timer()
    
    population = []
    
    def fitness(indiv):
        """Calculates the fitness function for the genetic algorithm""" 
        count = 0
        for i in range(len(indiv) - 1):
            count += (indiv[i] == np.array(indiv[i + 1:])).sum()

            upper_diagonal = indiv[i] + np.arange(1, queens - i)
            lower_diagonal = indiv[i] - np.arange(1, queens - i)
            count += (np.array(indiv[i + 1:]) == upper_diagonal).sum()
            count += (np.array(indiv[i + 1:]) == lower_diagonal).sum()

        fitness_score = ((queens*(queens-1))-(count*2))/2
            
        return fitness_score
        
    def is_goal(indiv):
        
        fitness_score = fitness(indiv)
        
        if fitness_score == (queens*(queens-1))/2:
            return True

        else:
            return False
        
    seed = 71 #460,530
    for i in range(n):
        game = EightQueensState(n = queens, seed = seed)
        population.append(game.call_array())
        seed +=1
    
    
    def reproduce(population):
        """
        Reproduction of the population:
            Tuple parents in pairs
            Crossover based on longest non-conflicting sequences
            Mutate based on a random probability
        Return new_population 
        """
        
        def non_conflict(parent):   
            """Finds the lenght of the non-conflicting sequence within a parent. """       
            index_row = 0
            index_diag = 0
            count=0

            for i in range(parent.size):

                if parent[i] in np.array(parent[:i]):
                    break
                else:
                    index_row = i

            for i in range(parent.size):

                upper_diagonal = (parent[i] + np.arange(1, (len(parent) - i)))[:i]
                lower_diagonal = (parent[i] - np.arange(1, len(parent) - i))[:i]
                
                count += np.count_nonzero(np.array(parent[:i]) == upper_diagonal)
                count += np.count_nonzero(np.array(parent[:i]) == lower_diagonal)
        
                if count == 1:
                    index_diag = i

                if bool(set(np.array(parent[:i])).intersection(upper_diagonal))== False and bool(set(np.array(parent[:i])).intersection(lower_diagonal))== False:
                    index_diag = i
                else:
                    break

            return min(index_row,index_diag)
        
        
        parents = []
        new_population = []
        
        while len(population)>0:
            x = population.pop(random.randrange(len(population)))
            y = population.pop(random.randrange(len(population)))
            parents.append((x,y)) 
        
        for i in range(len(parents)):
                
            index_par1 = non_conflict(parents[i][0])
            index_par2 = non_conflict(parents[i][1])
                
            
            swap_leng = max(index_par1, index_par2)
            if swap_leng <1:
                swap_leng = random. randint(1,queens-2)
            
     
            child1 = np.append(parents[i][0][:swap_leng+1],parents[i][1][swap_leng+1:])
            new_population.append(child1)
            child2 = np.append(parents[i][1][:swap_leng+1],parents[i][0][swap_leng+1:])
            new_population.append(child2)
            
        
            mutation_rate = 0.0001
            for child in new_population:
                if random.random() < mutation_rate:
                    mutation_index = random.randint(0,(queens-1))
                    child[mutation_index] = random.randint(swap_leng,(queens-1))
                         
        return new_population
    
    
    
    goal_state = [item for item in population if is_goal(item)== True]
    
    fitness_list = [(item, fitness(item)) for item in population]
    local_max = max(fitness_list , key = lambda i: i[1])[0]
    
    
    if len(goal_state) != 0:
        return goal_state[0]
    
    while len(goal_state)==0:
        
        ga_current = timeit.default_timer()
        time_total = ga_current-ga_start
        
        if time_total > 35:
            return f"Local maximum is found:{local_max} with fitness score {fitness(local_max)}."
        
        new_population = reproduce(population)
        population = new_population
        
        goal_state = [item for item in population if is_goal(item)== True]
        
        if len(goal_state) != 0:
            return goal_state[0].tolist()
        
        for i in range(len(population)):
            if fitness(population[i])> fitness(local_max):
                local_max = population[i]
            

In [6]:
for n in range(4,8):
    print(f"Solution for {n}-queens: {genetic_algorithm_nqueens(10, n)}")

Solution for 4-queens: [1, 3, 0, 2]
Solution for 5-queens: [3, 1, 4, 2, 0]
Solution for 6-queens: Local maximum is found:[3 5 0 4 1 4] with fitness score 14.0.
Solution for 7-queens: [5, 2, 0, 3, 6, 4, 1]
